In [8]:
import pandas as pd
import numpy as np

In [9]:
def read_data(file):
    df = pd.read_csv(file)
    df.head(10)
    return df

,OPEN_TIME,OPEN,HIGH,LOW,CLOSE,VOLUME,CLOSE_TIME,QUOTE_VOL,NUM_TRADES,TAKE_BASE_VOL,TAKE_QUOTE_VOL,EXTRA
0,1519496760000,0.085541,0.085570,0.085407,0.085473,27.714,1519496819999,2.370345,192,15.308,1.309733,0
1,1519496820000,0.085473,0.085570,0.085448,0.085449,20.128,1519496879999,1.720609,122,6.579,0.562654,0
2,1519496880000,0.085448,0.085569,0.085400,0.085538,45.510,1519496939999,3.890176,209,19.426,1.660960,0
3,1519496940000,0.085538,0.085546,0.085402,0.085540,22.231,1519496999999,1.900513,105,14.320,1.224483,0
4,1519497000000,0.085539,0.085552,0.085464,0.085464,15.243,1519497059999,1.303412,117,8.373,0.716219,0
5,1519497060000,0.085466,0.085555,0.085464,0.085545,31.895,1519497119999,2.727413,131,19.237,1.645547,0
6,1519497120000,0.085545,0.085545,0.085464,0.085539,13.325,1519497179999,1.139346,78,7.311,0.625352,0
7,1519497180000,0.085539,0.085555,0.085414,0.085517,28.202,1519497239999,2.411093,195,14.409,1.232346,0
8,1519497240000,0.085460,0.085545,0.085419,0.085534,15.302,1519497299999,1.307959,134,7.216,0.617035,0
9,1519497300000,0.085440,0.085545,0.085440,0.085533,13.730,1519497359999,1.173580,105,6.186,0.529001,0


In [3]:
# Target 1
def get_target_1(file, data=None):
    if isinstance(data, pd.DataFrame):
        df = data
    else:
        df = read_data(file)
    res = pd.DataFrame({})
    res['CLOSE_TIME'] = df.CLOSE_TIME
    res['TARGET']     = (df.OPEN > df.CLOSE).astype(int).shift(-1)
    return res.iloc[:-1,:]

def target_1_returns(data, signal):
    # data contains (atleast): OPEN_TIME, OPEN, CLOSE
    # signal : OPEN_TIME, ACTION {1: buy, 0: hold, -1: sell}
    
    data_signal_df = pd.merge(df_, signal, on='OPEN_TIME')
    data_signal_df.head()
    data_signal_df.shape
    
    return (data_signal_df.ACTION*(data_signal_df.CLOSE - data_signal_df.OPEN)).sum()

In [5]:
def eval_class_metrics(y_true, y_pred, thresh=0.5, metrics=[]):
    y_true = y_true.astype(int)
    y_scores = y_pred
    y_pred[y_pred >= thresh] = 1
    y_pred[y_pred < thresh]  = 0
    
    result = {}
    
    from sklearn.metrics import confusion_matrix
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    if 'tp' in metrics:
        result['tp'] = tp
    if 'fp' in metrics:
        result['fp'] = fp
    if 'tn' in metrics:
        result['tn'] = tn
    if 'fn' in metrics:
        result['fn'] = fn
    
    if 'acc' in metrics:
        from sklearn.metrics import accuracy_score
        result['acc'] = accuracy_score(y_true, y_pred)
    
    if 'f1' in metrics:
        from sklearn.metrics import f1_score
        result['f1'] = f1_score(y_true, y_pred)
    
    if 'auc' in metrics:
        from sklearn.metrics import roc_auc_score
        result['auc'] = roc_auc_score(y_true, y_scores)
    
    if 'log_loss' in metrics:
        from sklearn.metrics import log_loss
        result['log_loss'] = log_loss(y_true, y_pred)
    
    return result